In [1]:
import csv
from sklearn.model_selection import train_test_split
from functools import reduce


def calculate_output_shape(cfg):
    # const int output_h = (height + 2 * pad_h - (dilation_h * (kernel_h - 1) + 1)) / stride_h + 1;
    # const int output_w = (width + 2 * pad_w - (dilation_w * (kernel_w - 1) + 1)) / stride_w + 1;
    height, width = cfg['image_shape']
    kernel_h, kernel_w = cfg['kernel_shape']
    pad_h, pad_w = cfg['pads']
    dilation_h, dilation_w = cfg['dilations']
    stride_h, stride_w = cfg['strides']
    output_h = (height + 2 * pad_h - (dilation_h * (kernel_h - 1) + 1)) // stride_h + 1
    output_w = (width + 2 * pad_w - (dilation_w * (kernel_w - 1) + 1)) // stride_w + 1
    output_shape = [cfg['batch_size'], cfg['output_channels'], output_h, output_w]
    return output_shape


def calculate_gmacs(item):
    def reduce_list_prod(x):
        return reduce(lambda a, b : a * b, x)
    cfg = {
        'image_shape' : [int(item['H']), int(item['W'])],
        'kernel_shape' : [int(item['kH']), int(item['kW'])],
        'pads' : [int(item['pad_h']), int(item['pad_w'])],
        'dilations' : [int(item['dilation_h']), int(item['dilation_w'])],
        'strides' : [int(item['stride_h']), int(item['stride_w'])],
        'batch_size' : int(item['N']),
        'output_channels' : int(item['kN']),
    }
    output_shape = calculate_output_shape(cfg)
    gmacs = reduce_list_prod(output_shape) * reduce_list_prod(cfg['kernel_shape']) * int(item['C'])
    return gmacs * 1e-9


def data_processer(file_name : str, split : bool = False):
    feature, energy, latency = [], [], []
    with open(file_name, 'r') as fp:
        f_csv = csv.DictReader(fp)
        data = list(f_csv)
    for item in data:
        repeat = int(item['repeat'])
        feature_vec = [
                        # item['N'], 
                        # item['C'], 
                        # item['H'], 
                        item['W'], 
                        item['kN'], 
                        item['kC'], 
                        # item['kH'], 
                        item['kW'], 
                        # item['pad_h'], 
                        # item['pad_w'], 
                        # item['dilation_h'], 
                        # item['dilation_w'], 
                        item['stride_h'], 
                        # item['stride_w'], 
                        # item['group'], 
                        # item['has_bias'],
                        calculate_gmacs(item),
                    ]
        feature_vec = [int(x) for x in feature_vec]
        feature.append(feature_vec)
        energy.append(float(item['energy']) / repeat)
        latency.append(float(item['latency']) / repeat)
    if split:
        feature_train, feature_test, energy_train, energy_test, latency_train, latency_test, cfg_train, cfg_test = train_test_split(feature, energy, latency, data)
        train_split = {
            'feature' : feature_train,
            'energy' : energy_train,
            'latency' : latency_train,
            'raw_cfg' : cfg_train,
        }
        test_split = {
            'feature' : feature_test,
            'energy' : energy_test,
            'latency' : latency_test,
            'raw_cfg' : cfg_test,
        }
        return train_split, test_split
    else:
        train_data = {
            'feature' : feature,
            'energy' : energy,
            'latency' : latency,
            'raw_cfg' : data,
        }
        return train_data
train_data, val_data = data_processer('data/onnx_conv_resample_1w_results.csv', True)
test_data = data_processer('data/onnx_conv_raw_results.csv', False)
# pre-select test data
valid_test_idx = [i for i in range(len(test_data['raw_cfg'])) if test_data['raw_cfg'][i]['group'] == '1']
test_data = {
    'feature' : [test_data['feature'][i] for i in valid_test_idx],
    'energy' : [test_data['energy'][i] for i in valid_test_idx],
    'latency' : [test_data['latency'][i] for i in valid_test_idx],
    'raw_cfg' : [test_data['raw_cfg'][i] for i in valid_test_idx],
}
print(len(test_data['energy']))

918


In [2]:
import numpy as np
from sklearn.metrics import mean_squared_error


def get_accuracy(y_pred, y_true, threshold = 0.01):
    a = (y_true - y_pred) / y_true
    b = np.where(abs(a) <= threshold)
    return len(b[0]) / len(y_true)


def get_metrics(y_pred, y_true):
    rmspe = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100
    rmse = np.sqrt(mean_squared_error(y_pred, y_true))
    error = rmse / np.mean(y_true)
    acc5 = get_accuracy(y_pred, y_true, threshold=0.05)
    acc10 = get_accuracy(y_pred, y_true, threshold=0.10)
    acc15 = get_accuracy(y_pred, y_true, threshold=0.15)
    print(f"rmse: {rmse:.4f}; rmspe: {rmspe:.4f}; error: {error:.4f}; 5% accuracy: {acc5:.4f}; 10% accuracy: {acc10:.4f}; 15% accuracy: {acc15:.4f}.")


def large_error_indices(y_pred, y_true, threshold = 0.01):
    a = (y_true - y_pred) / y_true
    b = np.where(abs(a) > threshold)
    return b[0]

In [3]:
# RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
n_features = len(train_data['feature'][0])

large_error_items = []
for label in ["energy", "latency"]:
    model = RandomForestRegressor(
        max_depth=50,
        n_estimators=370,
        min_samples_leaf=1,
        min_samples_split=2,
        max_features=n_features,
        oob_score=True,
        random_state=10
    )
    model.fit(train_data["feature"], train_data[label])
    # val
    predicts = model.predict(val_data["feature"])
    print(f'{label} Results:')
    get_metrics(predicts, val_data[label])
    print("")
    # test
    predicts = model.predict(test_data["feature"])
    print('Testing set:')
    get_metrics(predicts, test_data[label])
    print("")
    # large error configs in train data
    indices_train = large_error_indices(model.predict(train_data['feature']), train_data[label], 0.15)
    items_train = [train_data['raw_cfg'][i] for i in indices_train]
    large_error_items += items_train
    # large error configs in val_data
    indices_val = large_error_indices(model.predict(val_data['feature']), val_data[label], 0.15)
    items_val = [val_data['raw_cfg'][i] for i in indices_val]
    large_error_items += items_val


energy Results:
rmse: 0.2351; rmspe: 22.0409; error: 0.5490; 5% accuracy: 0.4512; 10% accuracy: 0.6928; 15% accuracy: 0.8244.

Testing set:
rmse: 0.0359; rmspe: 88.2635; error: 1.1389; 5% accuracy: 0.3715; 10% accuracy: 0.6231; 15% accuracy: 0.7908.

latency Results:
rmse: 0.0026; rmspe: 24.1217; error: 0.4612; 5% accuracy: 0.4732; 10% accuracy: 0.7252; 15% accuracy: 0.8440.

Testing set:
rmse: 0.0007; rmspe: 80.1667; error: 1.0822; 5% accuracy: 0.4869; 10% accuracy: 0.7397; 15% accuracy: 0.8497.



In [4]:
import json
# save large error configs
save_config = False
print('total large error configs:', len(large_error_items))
if save_config:
    with open('data/large_error_items.csv', 'w') as f:
        f_csv = csv.DictWriter(f, large_error_items[0].keys())
        f_csv.writeheader()
        f_csv.writerows(large_error_items)

import sampling_conv as sc
# resample configs
nnmeter_format_configs = []
for item in large_error_items:
    c = {
        'HW' : int(item['H']),
        'CIN' : int(item['C']),
        'COUT' : int(item['kN']),
        'KERNEL_SIZE' : int(item['kH']),
        'STRIDES' : int(item['stride_h']),
    }
    nnmeter_format_configs.append(c)
ncfgs = sc.finegrained_sampling_conv(nnmeter_format_configs, 20)
# save resampled configs
fine_grained_configs = []
for c in ncfgs:
    conv_config = {
        # flexible params
        'image_shape' : [c['HW'], c['HW']],
        'input_channels' : c['CIN'],
        'kernel_channels' : c['CIN'],
        'output_channels' : c['COUT'],
        # constrained params
        'batch_size' : 1,
        'kernel_shape' : [c['KERNEL_SIZE'], c['KERNEL_SIZE']],
        'pads' : [c['KERNEL_SIZE'] // 2, c['KERNEL_SIZE'] // 2],
        'dilations' : [1, 1],
        'strides' : [c['STRIDES'], c['STRIDES']],
        'group' : 1,
        'has_bias' : True,
    }
    fine_grained_configs.append(conv_config)

if save_config:
    with open('data/fine_grained_configs.json', 'w') as f:
        json.dump(fine_grained_configs, f)


total large error configs: 1257


In [5]:
fine_grained_data = data_processer('data/onnx_conv_fine_grained_1w_results.csv', False)
# merge finegrained data into train data
for key in train_data.keys():
    train_data[key] += fine_grained_data[key]
# build predictor again
large_error_items = []
large_error_test_items = {}
for label in ["energy", "latency"]:
    model = RandomForestRegressor(
        max_depth=50,
        n_estimators=370,
        min_samples_leaf=1,
        min_samples_split=2,
        max_features=n_features,
        oob_score=True,
        random_state=10
    )
    model.fit(train_data["feature"], train_data[label])
    # val
    predicts = model.predict(val_data["feature"])
    print(f'{label} Results:')
    get_metrics(predicts, val_data[label])
    print("")
    # test
    predicts = model.predict(test_data["feature"])
    print('Testing set:')
    get_metrics(predicts, test_data[label])
    # large error items in test data
    indices_test = large_error_indices(model.predict(test_data['feature']), test_data[label], 0.15)
    items_test = [test_data['raw_cfg'][i] for i in indices_test]
    large_error_test_items[label] = items_test
    print("")
    # large error configs in train data
    indices_train = large_error_indices(model.predict(train_data['feature']), train_data[label], 0.15)
    items_train = [train_data['raw_cfg'][i] for i in indices_train]
    large_error_items += items_train
    # large error configs in val_data
    indices_val = large_error_indices(model.predict(val_data['feature']), val_data[label], 0.15)
    items_val = [val_data['raw_cfg'][i] for i in indices_val]
    large_error_items += items_val
    

energy Results:
rmse: 0.2447; rmspe: 10.3723; error: 0.5716; 5% accuracy: 0.5556; 10% accuracy: 0.8012; 15% accuracy: 0.9004.

Testing set:
rmse: 0.0472; rmspe: 100.0482; error: 1.4980; 5% accuracy: 0.4052; 10% accuracy: 0.6950; 15% accuracy: 0.8410.

latency Results:
rmse: 0.0023; rmspe: 10.1538; error: 0.4085; 5% accuracy: 0.5832; 10% accuracy: 0.8232; 15% accuracy: 0.9172.

Testing set:
rmse: 0.0010; rmspe: 102.0527; error: 1.4660; 5% accuracy: 0.5414; 10% accuracy: 0.8083; 15% accuracy: 0.9063.



In [6]:
print(len(train_data['feature']))

with open('data/failed_test_energy_items.csv', 'w') as f:
        f_csv = csv.DictWriter(f, large_error_test_items['energy'][0].keys())
        f_csv.writeheader()
        f_csv.writerows(large_error_test_items['energy'])

30774
